In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BlipProcessor, 
    BlipForConditionalGeneration,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset 
import transformers
from transformers import BlipProcessor, BlipForImageTextRetrieval,BlipForConditionalGeneration, AutoProcessor
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize
import os

import gc
import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm
import albumentations as A
import cv2
import shutil
import json
from PIL import Image
import requests
from matplotlib import pyplot as plt


c:\Users\isaac\anaconda3\envs\ai\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Users\isaac\anaconda3\envs\ai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from transformers import AutoProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
import time

class FastSurveillanceBLIPCaptioner:
    def __init__(self, model_path):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Load YOUR fine-tuned model
        self.model = BlipForConditionalGeneration.from_pretrained(
    model_path,
    torch_dtype=torch.float16,  # This keeps CUDA + reduces memory
    device_map="cuda"           # Automatic CUDA placement
)
        self.processor = AutoProcessor.from_pretrained(model_path)
        self.model.to(self.device).eval()
        
        # Warmup
        print("🔥 Warming up model...")
        dummy = Image.new('RGB', (384, 384), 'red')
        for _ in range(3):
            self._fast_caption(dummy)
        print("✅ Ready!")
    
    def _fast_caption(self, image):
        inputs = self.processor(
            images=image, 
            return_tensors="pt"
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,  # This already contains pad_token_id
                max_length=30,        # Good for surveillance detail
                num_beams=1,         # Fast greedy decoding
                do_sample=False,     
                early_stopping=True
                # Removed the duplicate pad_token_id!
            )
        
        return self.processor.decode(outputs[0], skip_special_tokens=True)
    
    def caption_single_image(self, image_path):
        if isinstance(image_path, str):
            image = Image.open(image_path).convert('RGB')
        else:
            image = image_path
            
        # Resize for optimal speed/quality balance
        image = image.resize((384, 384), Image.Resampling.LANCZOS)
        
        start = time.time()
        caption = self._fast_caption(image)
        time_ms = (time.time() - start) * 1000
        
        return {
            'caption': caption, 
            'inference_time_ms': time_ms
        }

# Test it:
captioner = FastSurveillanceBLIPCaptioner("./blip_surveillance_finetuned")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔥 Warming up model...


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ Ready!


In [17]:
!conda activate ai
!pip install -U bitsandbytes

In [11]:
import time

def test_inference_speed(captioner, image_paths):
    """Tests the inference speed of the captioner on a list of images."""
    
    print("🚀 Starting inference speed test...")
    
    total_time = 0
    num_images = len(image_paths)
    
    for i, image_path in enumerate(image_paths):
        try:
            # Use the captioner's built-in method
            result = captioner.caption_single_image(image_path)
            
            caption = result['caption']
            elapsed_time = result['inference_time_ms'] / 1000  # Convert to seconds
            total_time += elapsed_time
            
            print(f"Image {i+1}/{num_images}: {image_path} -> \"{caption}\" ({elapsed_time:.4f}s)")
            
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            num_images -= 1 # Adjust count if an image fails
            
    if num_images > 0:
        average_time = total_time / num_images
        print(f"\n✅ Test complete!")
        print(f"Average inference time: {average_time:.4f} seconds per image")
    else:
        print("\n❌ No images were processed.")

# Create sample image paths with proper path formatting
sample_images = [
    r"D:\keep\htx\fast_search\output_snapshots\screenshots\screenshot_2_f521bea5\scene_frame0_1750044961.jpg",
    r"D:\keep\htx\fast_search\output_snapshots\screenshots\screenshot_1_db28a20e\scene_frame0_1750042287.jpg",
    r"D:\keep\htx\fast_search\output_snapshots\person\person_id1_1750042287.jpg",
    r"D:\keep\htx\fast_search\output_snapshots\person\person_id709_1750045634.jpg"
]

# Run the test
test_inference_speed(captioner, sample_images)

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🚀 Starting inference speed test...


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Image 1/4: D:\keep\htx\fast_search\output_snapshots\screenshots\screenshot_2_f521bea5\scene_frame0_1750044961.jpg -> "the image is a full - scene screenshot showing an outdoor nighttime setting with a single bench visible in the foreground. there are no people or" (0.3911s)


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Image 2/4: D:\keep\htx\fast_search\output_snapshots\screenshots\screenshot_1_db28a20e\scene_frame0_1750042287.jpg -> "the image is a full - scene screenshot showing an outdoor nighttime scene with a street sign and traffic signs. there are no people or vehicles visible" (0.3456s)


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Image 3/4: D:\keep\htx\fast_search\output_snapshots\person\person_id1_1750042287.jpg -> "the image is a cropped object snapshot of a person standing in an outdoor area during the daytime. the person appears to be wearing a dark" (0.3795s)
Image 4/4: D:\keep\htx\fast_search\output_snapshots\person\person_id709_1750045634.jpg -> "the image is a full - scene screenshot showing a man wearing a black t - shirt and white pants walking outdoors during the daytime. he appears" (0.3406s)

✅ Test complete!
Average inference time: 0.3642 seconds per image
